SME0202 Métodos Numéricos em Equações Diferenciais Ordinárias
### Trabalho 1 - Equações Elípticas - Meios Anisotrópicos e Termo de Arraste
- Cody Stefano Barham Setti - 4856322
- Ian de Holanda Cavalcante Bezerra - 13835412

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
c = 4*np.pi**2 -3
# SOLUÇÃO EXATA DO PROBLEMA
def u(x,y):
    return np.exp(-y)*np.sin(2*np.pi*x)

In [ ]:
def print_matrix(matrix):
    # Pega a largura máxima precisa para qualquer elemento
    max_width = max(len(str(element)) for row in matrix for element in row)
    
    for line in matrix:
        # Formata cada elemento para que tenha largura consistente
        formatted_row = [str(element).rjust(max_width) for element in line]
        print(" ".join(formatted_row))

In [ ]:
# DISCRETIZAÇÃO DO DOMÍNIO
h = 0.25  # tamano de passo

x = np.arange(0,1+h, h)   # discretização do eixo-x
y = np.arange(-1,1+h, h)  # discretização do eixo-y

n = len(x)  # número de COLUNAS na malha
m = len(y)  # número de LINHAS da malha

print(n, m)

# Debug
print(f"x = {x}")
print(f"eixo-x discretizado em {n} pontos\n")
print(f"y = {y}")
print(f"eixo-y discretizado em {m} pontos")

5 9
x = [0.   0.25 0.5  0.75 1.  ]
eixo-x discretizado em 5 pontos

y = [-1.   -0.75 -0.5  -0.25  0.    0.25  0.5   0.75  1.  ]
eixo-y discretizado em 9 pontos


In [ ]:
# Indexação dos pontos da malha em DUAS coordenadas 
mesh = []
for i in range(m):
    line = []
    for j in range(n):
        line.append((int(i), int(j)))
    mesh.append(line)

print("Mudança da origem da malha para o canto superior esquerdo de Omega:")
print("--------------------------------------")
for point in mesh:
    for i, j in point:
        print(f"({i}, {j})", end="  ")  # 'Space' para formatação quadrada
    print()

print("--------------------------------------")
print(f"{m} linhas e {n} colunas")

Mudança da origem da malha para o canto superior esquerdo de Omega:
--------------------------------------
(0, 0)  (0, 1)  (0, 2)  (0, 3)  (0, 4)  
(1, 0)  (1, 1)  (1, 2)  (1, 3)  (1, 4)  
(2, 0)  (2, 1)  (2, 2)  (2, 3)  (2, 4)  
(3, 0)  (3, 1)  (3, 2)  (3, 3)  (3, 4)  
(4, 0)  (4, 1)  (4, 2)  (4, 3)  (4, 4)  
(5, 0)  (5, 1)  (5, 2)  (5, 3)  (5, 4)  
(6, 0)  (6, 1)  (6, 2)  (6, 3)  (6, 4)  
(7, 0)  (7, 1)  (7, 2)  (7, 3)  (7, 4)  
(8, 0)  (8, 1)  (8, 2)  (8, 3)  (8, 4)  
--------------------------------------
9 linhas e 5 colunas


In [ ]:
# Remoção dos bordos sob condição de Dirichlet - Recorte da malha
for i in range(m):
    line = mesh[i]
    line.remove(line[0])   # remoção do bordo esquerdo
    line.remove(line[-1])  # remoção do bordo direito

n-=2  # atualização do número de colunas

print("--------------------------------------")
for point in mesh:
    for i, j in point:
        print(f"({i}, {j})", end="  ")  # 'Space' para formatação quadrada
    print()

print("--------------------------------------")
print(f"{m} linhas e {n} colunas")

--------------------------------------
(0, 1)  (0, 2)  (0, 3)  
(1, 1)  (1, 2)  (1, 3)  
(2, 1)  (2, 2)  (2, 3)  
(3, 1)  (3, 2)  (3, 3)  
(4, 1)  (4, 2)  (4, 3)  
(5, 1)  (5, 2)  (5, 3)  
(6, 1)  (6, 2)  (6, 3)  
(7, 1)  (7, 2)  (7, 3)  
(8, 1)  (8, 2)  (8, 3)  
--------------------------------------
9 linhas e 3 colunas


In [ ]:
# Indexação dos pontos da malha em UMA coordenada

# Função auxiliar para conversão de indexação
def get_1d_index(i, j, n):
    return (j + i * n)

# Fazer um dicionário (i,j):k - armazenando, simultanemanete, índices 2D e 1D - muito mais elegante!!!
points = []
for i in range(m):
    for j in range(n):
        points.append(get_1d_index(i, j, n))

print("Enumeração Unidimensional da Malha:")
print("--------------------------------------")
flag = 0
for k in points:
    if (k<10):
        print(f"(  {k} )", end="  ")  # 'Space' para formatação quadrada
    else:
        print(f"( {k} )", end="  ")   # 'Space' para formatação quadrada
    if ((k+1)%n == 0): print()        # Atualiza linha do índice

print("--------------------------------------")
print(f"{m} linhas e {n} colunas")

Enumeração Unidimensional da Malha:
--------------------------------------
(  0 )  (  1 )  (  2 )  
(  3 )  (  4 )  (  5 )  
(  6 )  (  7 )  (  8 )  
(  9 )  ( 10 )  ( 11 )  
( 12 )  ( 13 )  ( 14 )  
( 15 )  ( 16 )  ( 17 )  
( 18 )  ( 19 )  ( 20 )  
( 21 )  ( 22 )  ( 23 )  
( 24 )  ( 25 )  ( 26 )  
--------------------------------------
9 linhas e 3 colunas


In [ ]:
# DISCRETIZAÇÃO DA EQUAÇÃO ELÍPTICA
int_points = n*m                        # número de incógnitas
A = np.zeros((int_points, int_points))  # inicialização do operador diferencial discretizado
F = np.zeros(int_points)                # inicialização do vetor de forçamento
c = 4*np.pi**2 -3                    # constante de arraste vertical
print(f"C is: {c}")


# Percorrendo os pontos (i,j) da malha (de incógnitas) para construir-se o sistema AU=F
k = 0  # índice da incógnita. Isto é: U0, U1, U2, ... , Un*m
for i in range(0, m):
    for j in range(0, n):  # Note: pontos sob condição de Dirichlet ignorados

        # Incógnitas no bordo superior da malha
        if i == 0:
            A[k][k] = 4
            A[k][get_1d_index(i, j-1, n)] = -4
            A[k][get_1d_index(i, j-2, n)] = 1

        # Incógnitas no bordo inferior da malha
        if i == m-1:
            A[k][k] = -3
            A[k][get_1d_index(i-1, j, n)] = 4
            A[k][get_1d_index(i-2, j, n)] = -1

            F[k] = 2*np.pi*np.exp(1)*np.cos(2*np.pi*x[j])

        # Incógnitas do interior da malha
        else:
            A[k][k] = -4
            A[k][get_1d_index(i+1, j, n)] = 1
            A[k][get_1d_index(i-1, j, n)] = 1
            if j != 0: A[k][get_1d_index(i, j-1, n)] = 3+((c*h)/2)  # Se não está no bordo esquerdo 
            if j != n: A[k][get_1d_index(i, j+1, n)] = 3-((c*h)/2)  # Se não está no bordo direito

        k += 1

# Solução numérica da Eq. Elíptica
U = np.linalg.solve(A,F)

# Debug
# print_matrix(A)
print(f"Dimensão de A: {len(A)}x{len(A[0])}")
print("U = ",U)

C is: 36.47841760435743
Dimensão de A: 27x27
U =  [-5.75955700e+00 -5.82963047e+00 -1.21296162e+01 -1.02539452e+00
 -1.21454717e+00 -5.65371269e+00 -2.36474414e-01 -9.54519589e-02
 -1.67235153e+00 -6.93893145e-02  1.19015317e-02 -4.22329109e-01
 -2.25188085e-02  8.87770466e-03 -1.42063018e-01 -6.83845624e-03
 -2.77431823e-02 -2.23703291e-01 -4.81088932e-02 -4.17085944e-01
 -6.18057535e-01 -8.36168689e-01 -2.24095438e+00 -3.99697377e-01
 -6.79201144e+00 -2.84891052e+00  5.36624549e+00]


### Tarefa 1: Obter solução numérica

### Tarefa 2: Obter solução analítica + cálculo dos erros

In [ ]:
# Solucao analitica
def u(x,y):
    return np.exp(-y)*np.sin(2*np.pi*x)

# Calculo dos erros
def erro(U, u):
    return np.max(np.abs(U - u))







# Reshape U into 2D grid for plotting
U_2d = U.reshape((m, n+1))

# Create meshgrid for analytical solution
X, Y = np.meshgrid(x, y)
U_analytical = u(X, Y)

# Create figure with 3D subplots
fig = plt.figure(figsize=(12, 5))

# Numerical solution plot
ax1 = fig.add_subplot(121, projection='3d')
surf1 = ax1.plot_surface(X, Y, U_2d, cmap='viridis')
ax1.set_xlabel('x')
ax1.set_ylabel('y')
ax1.set_zlabel('u')
ax1.set_title('Numerical Solution')
fig.colorbar(surf1, ax=ax1)

# Analytical solution plot
ax2 = fig.add_subplot(122, projection='3d')
surf2 = ax2.plot_surface(X, Y, U_analytical, cmap='viridis')
ax2.set_xlabel('x')
ax2.set_ylabel('y')
ax2.set_zlabel('u')
ax2.set_title('Analytical Solution')
fig.colorbar(surf2, ax=ax2)

plt.tight_layout()
plt.show()

# Calculate and print maximum error
max_error = erro(U_2d, U_analytical)
print(f"Maximum absolute error: {max_error}")


ValueError: cannot reshape array of size 27 into shape (9,4)

### Tarefa 3: Estudo de convergência do método numérico